In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%mkdir yolov5_ws
%cd yolov5_ws
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

/content/yolov5_ws
Cloning into 'yolov5'...
remote: Enumerating objects: 16941, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 16941 (delta 70), reused 91 (delta 48), pack-reused 16805 (from 1)
Receiving objects: 100% (16941/16941), 15.70 MiB | 16.76 MiB/s, done.
Resolving deltas: 100% (11603/11603), done.
/content/yolov5_ws/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.1/872.1 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 936.3 kB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


In [ ]:
import cv2
import torch
import numpy as np

def preprocess_frame(frame, size=(640, 640)):
    # Resize frame to the target size and keep aspect ratio
    return cv2.resize(frame, size)

def postprocess_boxes(boxes, original_shape, resized_shape):
    # Rescale bounding boxes to match the original frame size
    original_height, original_width = original_shape
    resized_height, resized_width = resized_shape

    boxes_rescaled = []
    for box in boxes:
        x1, y1, x2, y2, conf, cls = box
        # Calculate scaling factors
        x_scale = original_width / resized_width
        y_scale = original_height / resized_height
        # Rescale box coordinates
        x1 = int(x1 * x_scale)
        y1 = int(y1 * y_scale)
        x2 = int(x2 * x_scale)
        y2 = int(y2 * y_scale)
        boxes_rescaled.append([x1, y1, x2, y2, conf, cls])
    return boxes_rescaled

def run_inference_on_video(model_path, input_video, output_video, conf_threshold=0.1):
    # Load YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

    # Set model confidence threshold
    model.conf = conf_threshold

    # Open the input video file
    video_capture = cv2.VideoCapture(input_video)
    if not video_capture.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    original_frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))

    # Define codec and create VideoWriter object
    video_writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, (original_frame_width, original_frame_height))

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        # Preprocess frame (resize to 640x640)
        resized_frame = preprocess_frame(frame)

        # Perform inference
        results = model(resized_frame)

        # Get bounding boxes and annotations
        boxes = results.xyxy[0].cpu().numpy()  # Get boxes as numpy array

        # Post-process boxes to original size
        boxes_rescaled = postprocess_boxes(boxes, (original_frame_height, original_frame_width), (640, 640))

        # Draw bounding boxes on the original frame
        for box in boxes_rescaled:
            x1, y1, x2, y2, conf, cls = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f'{model.names[cls]} {conf:.2f}'
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Write the processed frame to the output video
        video_writer.write(frame)

    # Release video objects
    video_capture.release()
    video_writer.release()

    print(f"Output video saved to: {output_video}")

# Define paths to model, input video, and output video
model_path = '/content/destilation_prunned_quantized_fixed.onnx'  # Path to the YOLOv5 model (PyTorch format)
input_video_path = '/content/test_video.mp4'  # Path to your input video
output_video_path = '/content/output.mp4'  # Path to save the output video

# Set a lower confidence threshold (e.g., 0.2)
run_inference_on_video(model_path, input_video_path, output_video_path, conf_threshold=0.4)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-9-4 Python-3.10.12 torch-2.4.0+cu121 CPU

Loading /content/destilation_prunned_quantized_fixed.onnx for ONNX Runtime inference...
Adding AutoShape... 
